ЗАДАНИЯ С БАЗОЙ ДАННЫХ:
1. Вытащите данные и найдите средний возраст клиентов из Минска
2. Выведите список всех клиентов с балансом больше 1000
3. Подсчитайте количество транзакций по каждому типу
4. Определите клиентов с наибольшими расходами (отрицательные суммы транзакций)
5. Топ-3 клиента по количеству транзакций
6. Выявление подозрительных операций, операции > n% от баланса

In [10]:
import pandas as pd
import sqlite3

with sqlite3.connect('database.db') as conn:
    # 1 Выбираем все столбцы из таблицы клиентов
    df_cl = pd.read_sql('SELECT * FROM clients', conn)

    # Средний возраст клиентов из Минска
    average_age = df_cl[df_cl['city'] == 'Минск']['age'].mean()
    print(f'Средний возраст клиентов из Минска: {average_age}')

    # Альтернативный способ через SQL-запрос
    df = pd.read_sql('SELECT age FROM clients WHERE city = "Минск"', conn)
    average_age_sql = df.mean()

    # 2 Выбираем все столбцы из таблицы баланс
    df_b = pd.read_sql('SELECT * FROM balance', conn)

    # Фильтруем данные
    balance100 = df_b[df_b['amount'] > 1000]
    print('\n Список клиентов с балансом больше 1000')
    print(balance100)

    # Альтернативный SQL-запрос
    df = pd.read_sql('SELECT client_id, amount FROM balance WHERE amount > 1000', conn)

    # 3 Выбираем все столбцы из таблицы транзакций
    df_t = pd.read_sql('SELECT * FROM transactions', conn)

    # Количество уникальных значений в столбце 'type'
    unique_transactions = df_t['type'].value_counts()
    print('\nКоличество транзакций по каждому типу')
    print(unique_transactions)

    # Альтернативный SQL-запрос
    df = pd.read_sql('SELECT type FROM transactions', conn)
    unique_transactions_sql = df.value_counts()

    # 4 Фильтруем отрицательные суммы транзакций
    negative_transactions = df_t[df_t['amount'] < 0]
    print('\nСписок клиентов с наибольшими расходами')
    print(negative_transactions)

    # Альтернативный SQL-запрос
    df = pd.read_sql('SELECT * FROM transactions WHERE amount < 0', conn)

    # 5 Топ-3 клиентов по количеству транзакций
    top_3 = df_t['client_id'].value_counts().head(3)
    print('\nТоп-3 клиентов по количеству транзакций')
    print(top_3)

    # Альтернативный SQL-запрос
    df = pd.read_sql('SELECT client_id FROM transactions', conn)
    top_3_sql = df.value_counts().head(3)
    

    # 6 Объединяем DataFrame баланса и транзакций по ключу client_id
    merge_df = df_b.merge(df_t, on='client_id')

    # Устанавливаем порог процент от баланса
    n = int(input()) 

    # Добавляем столбец и высчитываем порог транзакции
    merge_df['transaction_threshold'] = merge_df['amount_x'] * (n/100)

    # Фильтруем транзакции, которые превышают порог
    suspicious_transactions = merge_df[merge_df['amount_y'] > merge_df['transaction_threshold']]

    print('\nПодозрительные операции')
    print(suspicious_transactions)

Средний возраст клиентов из Минска: 31.5

 Список клиентов с балансом больше 1000
   client_id  amount
1          2    1500
3          4    2000
4          5    1200

Количество транзакций по каждому типу
type
deposit       5
withdrawal    5
Name: count, dtype: int64

Список клиентов с наибольшими расходами
   transaction_id  client_id        date  amount        type     description
1               2          1  2025-03-02     -50  withdrawal  ATM withdrawal
3               4          2  2025-03-03     -30  withdrawal   Grocery store
5               6          3  2025-03-04     -20  withdrawal     Coffee shop
7               8          4  2025-03-05     -70  withdrawal      Restaurant
9              10          5  2025-03-06     -40  withdrawal        Shopping

Топ-3 клиентов по количеству транзакций
client_id
1    2
2    2
3    2
Name: count, dtype: int64

Подозрительные операции
   client_id  amount_x  transaction_id        date  amount_y     type  \
0          1      1000           